# MNIST Character Recognition using Deep Learning

* First notebook on Kaggle.

* The following implementation uses a Convolutional Neural Network for Handwritten Character Recognition on the MNIST Dataset.

* More detailed information about the codes are in respective markdowns.

In [ ]:
#Importing libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Sequential

from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

# Data Preparation

### The `train.csv` and `test.csv` files are first loaded into `training` and `testing` variables using Pandas.

In [ ]:
#Loading data

training = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
testing = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')



### The dataset is divided into the labels and data - pixel values of each image and converted to **numpy arrays** for easier reshaping further in the code.

In [ ]:
#Dividing data and corresponding label into x_train and y_train
trainingLabels = training['label']
trainingData = training.drop(labels=['label'], axis=1)

#Converting data to numpy array
trainingLabels, trainingData = trainingLabels.values, trainingData.values

### Normalization is performed on the pixel values to bring them in a range of [0, 1]. Normalization is done as a CNN converges much faster and easier with smaller values.

In [ ]:
#Normalizing Data

trainingData = trainingData / 255.0
testing = testing / 255.0

### Image data is reshaped to image in 3 dimensions - height 28px width 28px channel = 1

In [ ]:
#Reshape data to image in 3 dimensions
trainingData = trainingData.reshape(-1, 28, 28, 1)
testing = testing.values.reshape(-1, 28, 28, 1)

### Each label is converted to a One Hot Vector of 10 classes where 1 represents the class a digit belongs to. 

Example - For the number 2, the one hot vector will be [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
#One Hot Encoding
trainingLabels = to_categorical(trainingLabels, num_classes = 10)

### Training Data is split into training and validation data in the ratio of 9:1 

In [ ]:
#Splitting data into training and validation datasets (9:1)
x_train, x_val, y_train, y_val = train_test_split(trainingData, trainingLabels, test_size = 0.1, random_state=2)

## Model Building

* The CNN Architecture consists of 2 Convolutional 2D Layers with 64 and 32 filters respectively and a kernel size of 3.
* Each Conv2D layer is followed by a MaxPooling Layer of size 2
* Dropout of 0.25 after each set of Conv2D and MaxPool Layer prevents overfitting by disconnecting neurons in the network.
* The final two layers are a Flatten Layer - to flatten data into a single dimension vector and an output dense layer with a 'softmax' activation function and 10 classes.

In [ ]:
#Building CNN Model
model = Sequential()

#First Layer
model.add(Conv2D(64, 3, activation = 'relu', input_shape = (28, 28, 1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#Second Layer
model.add(Conv2D(32, 3, activation = 'relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

#Final Output Layer
model.add(Flatten())
model.add(Dense(10, activation = 'softmax'))

## Compiling Model 

The 'Adam' Optimizer was used to compile the model and loss function was a 'categorical_crossentropy'

In [ ]:
#Compiling Model
model.compile(optimizer='adam' , loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
#Training Model 
history = model.fit(x_train, y_train, batch_size = 64, epochs = 10, 
                    validation_data = (x_val, y_val), verbose = 2)

In [ ]:
#Visualizing Model Accuracy

plt.title('Model Accuracy')
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Val'])
plt.show()

In [ ]:
predictions = model.predict(testing[:3])
#Printing prediction of model with labels of first 3 test images
print(np.argmax(predictions, axis = 1))

In [ ]:
def prep_test_data(raw):
    x = raw[:,0:]
    num_images = raw.shape[0]
    out_x = x.reshape(num_images, 28, 28, 1)
    out_x = out_x / 255
    return out_x

val_file = "/kaggle/input/digit-recognizer/test.csv"
val_data = np.loadtxt(val_file, skiprows=1, delimiter=',')
x_test = prep_test_data(val_data)

In [ ]:
predictions = model.predict_classes(x_test)

indexes = [i for i in range(1,len(val_data)+1)]
output = pd.DataFrame({'ImageId': indexes,'Label': predictions})
output.to_csv('submission.csv', index=False)

In [ ]:
pd.read_csv('submission.csv')